# Arima

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import itertools

# Carregar o arquivo principal (dados históricos até 2023)
data = pd.read_csv(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Forecast\Arquivos\data.csv")
data['Date mais recente'] = pd.to_datetime(data['Date mais recente'])
data.set_index('Date mais recente', inplace=True)
ts_data = data['Total']

# Definir as faixas de valores a serem testados para (p, d, q)
p = d = q = range(0, 4)  # Testar valores de 0 a 2 para cada parâmetro
pdq_combinations = list(itertools.product(p, d, q))

# Variáveis para armazenar o melhor modelo e o menor AIC
best_aic = float("inf")
best_order = None
best_model = None

# Loop para encontrar a melhor combinação de (p, d, q)
for order in pdq_combinations:
    try:
        # Ajustar o modelo ARIMA com a combinação atual
        model = ARIMA(ts_data, order=order)
        fitted_model = model.fit()
        # Verificar se o AIC do modelo atual é o menor encontrado
        if fitted_model.aic < best_aic:
            best_aic = fitted_model.aic
            best_order = order
            best_model = fitted_model
    except Exception as e:
        # Ignorar combinações que causam erros
        continue

# Exibir a melhor combinação de (p, d, q) e o valor de AIC correspondente
print(f"Melhor ordem (p, d, q): {best_order}")
print(f"Menor AIC: {best_aic}")

# Opcional: Exibir o resumo do melhor modelo encontrado
print(best_model.summary())


In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Carregar o arquivo principal (dados históricos até 2023)
data = pd.read_csv(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Forecast\Arquivos\data.csv")
data['Date mais recente'] = pd.to_datetime(data['Date mais recente'])
data.set_index('Date mais recente', inplace=True)
ts_data = data['Total']

# Carregar o arquivo de saída para 2024 (dados reais de 2024)
data_2024 = pd.read_csv(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\2024 saidas.csv")
data_2024['Date mais recente'] = pd.to_datetime(data_2024['Date mais recente'])
data_2024.set_index('Date mais recente', inplace=True)
ts_data_2024 = data_2024['Total']

# Ajustar o modelo ARIMA nos dados históricos até o final de 2023
model_full = ARIMA(ts_data, order=(2, 2, 1))  # Parâmetros (p, d, q) iniciais
fitted_model_full = model_full.fit()

# Fazer previsões para o ano de 2024 (12 meses à frente)
forecast_2024 = fitted_model_full.forecast(steps=12)

# Plotar os dados históricos, os dados reais de 2024 e a previsão para 2024
plt.figure(figsize=(12, 6))
plt.plot(ts_data, label='Histórico até 2023')
plt.plot(ts_data_2024, label='Real 2024')
plt.plot(forecast_2024, label='Previsão ARIMA para 2024', linestyle='--')
plt.title('Comparação da Previsão ARIMA para 2024 com os Dados Reais')
plt.legend()
plt.show()


comparison_df = pd.DataFrame({
    'Real 2024': ts_data_2024,
    'Previsão ARIMA 2024': forecast_2024
})
# Exibir a tabela comparativa
print(comparison_df)


# ExponentialSmoothing

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt

monthly_data = pd.read_csv(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Forecast\Arquivos\data.csv")

# Selecionar a série temporal mensal
monthly_series = monthly_data['Total']

# Ajustar o modelo ETS com sazonalidade anual (período sazonal de 12 meses)
model = ExponentialSmoothing(monthly_series, 
                             trend='add', 
                             seasonal='add', 
                             seasonal_periods=12)
model_fit = model.fit()

# Previsão para os próximos 12 meses
forecast = model_fit.forecast(steps=12)


# Adicionar rótulos nos pontos dos dados históricos
forecast_df = forecast.to_frame(name='Previsão_Qtd_UM_registro')

# Exibir o DataFrame de previsão
forecast_df

In [ ]:
monthly_data.head

# LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Carregar e pré-processar os dados
def load_and_preprocess_data(file_path, seq_length):
    data = pd.read_csv(file_path)
    data_values = data[['Total']].values  # Supondo que a coluna a ser usada é 'Total'
    
    # Normalizar os dados
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data_values)
    
    # Função para criar sequência de dados para o modelo LSTM
    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:i + seq_length])
            y.append(data[i + seq_length])
        return np.array(X), np.array(y)

    X, y = create_sequences(data_scaled, seq_length)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    return X_train, X_test, y_train, y_test, scaler, data_scaled

# Construir o modelo LSTM
def build_lstm_model(seq_length):
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(seq_length, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Treinar o modelo com Early Stopping
def train_model(model, X_train, y_train, X_test, y_test):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test),
                        callbacks=[early_stopping], batch_size=16, verbose=0)
    return history

# Avaliação e visualização dos resultados
def evaluate_and_plot(model, X_test, y_test, scaler):
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler.inverse_transform(y_pred_scaled)
    y_test_original = scaler.inverse_transform(y_test)
    
    plt.plot(y_test_original, label='Valor Real')
    plt.plot(y_pred, label='Previsão')
    plt.xlabel('Tempo')
    plt.ylabel('Demanda')
    plt.legend()
    plt.show()
    
    # Formatar resultados no padrão brasileiro
    results = pd.DataFrame({
        'Valor Real': [f"{val[0]:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") for val in y_test_original],
        'Previsão': [f"{val[0]:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") for val in y_pred]
    })
    return results

# Função para previsão de meses futuros
def forecast_future(model, scaler, data_scaled, seq_length, n_months):
    last_sequence = data_scaled[-seq_length:]
    predictions = []
    
    for _ in range(n_months):
        last_sequence_reshaped = last_sequence.reshape((1, seq_length, 1))
        next_pred_scaled = model.predict(last_sequence_reshaped)
        next_pred = scaler.inverse_transform(next_pred_scaled)
        predictions.append(next_pred[0, 0])
        last_sequence = np.append(last_sequence, next_pred_scaled).reshape(-1, 1)
        last_sequence = last_sequence[1:]

    # Formatar previsões no padrão brasileiro
    dates_2024 = pd.date_range(start='2024-01-01', periods=n_months, freq='M')
    forecast_2024 = pd.DataFrame({
        'Data': dates_2024,
        'Previsão de Demanda': [f"{val:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") for val in predictions]
    })
    return forecast_2024

# Configurações principais
file_path = r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\data.csv"
seq_length = 12
n_months_to_predict = 12

# Execução completa
X_train, X_test, y_train, y_test, scaler, data_scaled = load_and_preprocess_data(file_path, seq_length)
model = build_lstm_model(seq_length)
train_model(model, X_train, y_train, X_test, y_test)
results = evaluate_and_plot(model, X_test, y_test, scaler)
forecast_2024 = forecast_future(model, scaler, data_scaled, seq_length, n_months_to_predict)

# Exibir previsões de 2024
forecast_2024

